Generative film notebook 001


Aims :

- grab 3 x random videos from youtube 
- cut into small pieces and concatenate with one another to create a new film

- add search parameters
- add 




In [4]:

# fix for ImageMagick bug from https://github.com/Zulko/moviepy/issues/347
from moviepy.config import change_settings

In [5]:
# found this path using brew info imageMagick
change_settings({"IMAGEMAGICK_BINARY": "/opt/homebrew/Cellar/imagemagick/7.1.0-27/bin/convert"})


In [6]:
import os
import sys

import numpy as np

from moviepy import *
from moviepy.video import *

from moviepy.audio.tools.cuts import find_audio_period
from moviepy.video.tools.cuts import find_video_period

from moviepy.audio.AudioClip import *
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, concatenate_videoclips, clips_array, vfx, CompositeVideoClip






In [7]:


# Next lines are for downloading the required videos from Youtube.
# To do this you must have youtube-dl installed, otherwise you will need to
# download the videos by hand and rename them, as follows:
#     https://www.youtube.com/watch?v=zvCvOC2VwDc => knights.mp4
#     https://www.youtube.com/watch?v=lkY3Ek9VPtg => frontier.mp4

if not os.path.exists("knights.mp4") and not os.path.exists("frontier.webm"):
    retcode1 = os.system("youtube-dl zvCvOC2VwDc -o knights")
    retcode2 = os.system("youtube-dl lkY3Ek9VPtg -o frontier")
    if retcode1 != 0 or retcode2 != 0:
        sys.stderr.write(
            "Error downloading videos. Check that you've installed youtube-dl.\n"
        )
        sys.exit(1)

In [8]:
# ==========


# LOAD, EDIT, ANALYZE THE AUDIO

audio = (
    AudioFileClip("frontier.webm")
    .subclip((1, 7), (1, 18))
    .audio_fadein(1)
    .audio_fadeout(1)
)

audio_period = find_audio_period(audio)
print("Analyzed the audio, found a period of %.02f seconds" % audio_period)


Analyzed the audio, found a period of 0.72 seconds


In [9]:
# LOAD, EDIT, ANALYZE THE VIDEO

clip = (
    VideoFileClip("knights.mp4", audio=False)
    .subclip((1, 24.15), (1, 26))
    .crop(x1=500, x2=1350)
)

video_period = find_video_period(clip, tmin=.3)
print("Analyzed the video, found a period of %.02f seconds" % video_period)

edited_right = (
    clip.subclip(0, video_period)
    .speedx(final_duration=2 * audio_period)
    .fx(vfx.loop, duration=audio.duration)
    .subclip(0.25)
)

edited_left = edited_right.fx(vfx.mirror_x)



Analyzed the video, found a period of 1.23 seconds


In [10]:

clips = [[edited_left, edited_right]]
final_video = clips_array(clips)

dancing_knights = (
    final_video.fadein(1).fadeout(1).set_audio(audio).subclip(.3)
)

In [11]:
# MAKE THE TITLE SCREEN

txt_title = (TextClip("15th century dancing\n(hypothetical)", fontsize=70,
               font="Century-Schoolbook-Roman", color="white")
             .margin(top=15, opacity=0)
             .set_position(("center","top")))



In [12]:
title = (CompositeVideoClip([dancing_knights.to_ImageClip(), txt_title])
         .fadein(.5)
         .set_duration(3.5))




In [13]:

# MAKE THE CREDITS SCREEN


txt_credits = """
CREDITS

Video excerpt: Le combat en armure au XVe siècle
By J. Donzé, D. Jaquet, T. Schmuziger,
Université de Genève, Musée National de Moyen Age

Music: "Frontier", by DOCTOR VOX
Under licence Creative Commons
https://www.youtube.com/user/DOCTORVOXofficial

Video editing © Zulko 2014
 Licence Creative Commons (CC BY 4.0)
Edited with MoviePy: http://zulko.github.io/moviepy/
"""

credits = (TextClip(txt_credits, color='white',
            font="Century-Schoolbook-Roman", fontsize=35, kerning=-2,
            interline=-1, bg_color='black', size=title.size)
          .set_duration(2.5)
          .fadein(.5)
          .fadeout(.5))





In [14]:
# ASSEMBLE EVERYTHING, WRITE TO FILE

final = concatenate_videoclips([title, dancing_knights, credits])


final.write_videofile("dancing_knights0.mp4",
                      fps=clip.fps,
                      audio_bitrate="1000k", 
                      bitrate="4000k",
                     codec='libx264', 
                     audio_codec='aac', 
                     )

Moviepy - Building video dancing_knights.mp4.
MoviePy - Writing audio in dancing_knightsTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video dancing_knights.mp4



Moviepy - Done !
Moviepy - video ready dancing_knights.mp4


===================================================================================================

NOW MY OWN VERSION WITH BOWIE

In [20]:


import os

from moviepy.audio.tools.cuts import find_audio_period
from moviepy.editor import *
from moviepy.video.tools.cuts import find_video_period

# Next lines are for downloading the required videos from Youtube.
# To do this you must have youtube-dl installed, otherwise you will need to
# download the videos by hand and rename them, as follows:
#     https://www.youtube.com/watch?v=zvCvOC2VwDc => knights.mp4
#     https://www.youtube.com/watch?v=lkY3Ek9VPtg => frontier.mp4


# this doesnt work atm - even if frontier added, its not an mp4 and audio silent on final file


if not os.path.exists("bowie.mp3"):
    os.system("youtube-dl zvCvOC2VwDc -o knights.mp4")
    os.system("youtube-dl -x --audio-format mp3 WoDamvrfUbQ -o BOWIE.mp3")
    
#==========


[youtube] zvCvOC2VwDc: Downloading webpage
[download] knights.mp4 has already been downloaded and merged
[youtube] WoDamvrfUbQ: Downloading webpage
[download] Destination: BOWIE.mp3
[download] 100% of 2.84MiB in 00:5701KiB/s ETA 00:00
[ffmpeg] Correcting container in "BOWIE.mp3"
[ffmpeg] Post-process file BOWIE.mp3 exists, skipping


In [21]:


# LOAD, EDIT, ANALYZE THE AUDIO


# Had to edit this to use with alternative music - 
# subclip(self, t_start=0, t_end=None)
# Returns a clip playing the content of the current clip between times t_start and t_end,
# which can be expressed in seconds (15.35), in (min, sec), in (hour, min, sec)

audio = (AudioFileClip("bowie.mp3")
         .subclip((1,7), (1,18))
         .audio_fadein(1)
         .audio_fadeout(1))



audio_period = find_audio_period(audio)
print ('Analyzed the audio, found a period of %.02f seconds'%audio_period)


# LOAD, EDIT, ANALYZE THE VIDEO


clip = (VideoFileClip("./knights.mp4", audio=False)
        .subclip((1,24.15),(1,26))
        .crop(x1=500, x2=1350))

video_period = find_video_period(clip, tmin=.3)
print ('Analyzed the video, found a period of %.02f seconds'%video_period)

edited_right = (clip.subclip(0,video_period)
                .speedx(final_duration=2*audio_period)
                .fx(vfx.loop, duration=audio.duration)
                .subclip(.25))

edited_left = edited_right.fx(vfx.mirror_x)

dancing_knights = (clips_array([[edited_left, edited_right]])
                   .fadein(1).fadeout(1).set_audio(audio).subclip(.3))

# MAKE THE TITLE SCREEN


txt_title = (TextClip("15th century dancing\n(hypothetical)", fontsize=70,
               font="Century-Schoolbook-Roman", color="white")
             .margin(top=15, opacity=0)
             .set_position(("center","top")))

title = (CompositeVideoClip([dancing_knights.to_ImageClip(), txt_title])
         .fadein(.5)
         .set_duration(3.5))


# MAKE THE CREDITS SCREEN


txt_credits = """
CREDITS

Video excerpt: Le combat en armure au XVe siècle
By J. Donzé, D. Jaquet, T. Schmuziger,
Université de Genève, Musée National de Moyen Age

Music: by BOWIE
Under licence Creative Commons
https://www.youtube.com/user/DOCTORVOXofficial

Video editing © SAM 2022
 Licence Creative Commons (CC BY 4.0)
Edited with MoviePy: http://zulko.github.io/moviepy/
"""

credits = (TextClip(txt_credits, color='white',
            font="Century-Schoolbook-Roman", fontsize=35, kerning=-2,
            interline=-1, bg_color='black', size=title.size)
          .set_duration(2.5)
          .fadein(.5)
          .fadeout(.5))


# ASSEMBLE EVERYTHING, WRITE TO FILE

final = concatenate_videoclips([title, dancing_knights, credits])


final.write_videofile("swinging_knights.mp4",
                      fps=clip.fps,
                      audio_bitrate="1000k", 
                      bitrate="4000k",
                     codec='libx264', 
                     audio_codec='aac'
                     )

Analyzed the audio, found a period of 0.29 seconds
Analyzed the video, found a period of 1.23 seconds
Moviepy - Building video swinging_knights.mp4.
MoviePy - Writing audio in swinging_knightsTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video swinging_knights.mp4



Moviepy - Done !
Moviepy - video ready swinging_knights.mp4


======================================================================================================

MORE EXPERIMENTS

In [4]:
# Searching for a specific term (outputting the first 2 videos)

os.system("youtube-dl 'ytsearch2:ducklings' ")




[download] Downloading playlist: ducklings
[youtube:search] query "ducklings": Downloading page 1
[youtube:search] playlist ducklings: Downloading 2 videos
[download] Downloading video 1 of 2
[youtube] e90eWYPNtJ8: Downloading webpage


[download] Destination: Cute Duckling 🔴 Funniest Baby Ducks Videos Compilation 2020-e90eWYPNtJ8.f137.mp4
[download] 100% of 158.02MiB in 34:1864KiB/s ETA 00:00500
[download] Destination: Cute Duckling 🔴 Funniest Baby Ducks Videos Compilation 2020-e90eWYPNtJ8.f251.webm
[download] 100% of 9.71MiB in 03:5931KiB/s ETA 00:00:13
[ffmpeg] Merging formats into "Cute Duckling 🔴 Funniest Baby Ducks Videos Compilation 2020-e90eWYPNtJ8.mkv"
Deleting original file Cute Duckling 🔴 Funniest Baby Ducks Videos Compilation 2020-e90eWYPNtJ8.f137.mp4 (pass -k to keep)
Deleting original file Cute Duckling 🔴 Funniest Baby Ducks Videos Compilation 2020-e90eWYPNtJ8.f251.webm (pass -k to keep)
[download] Downloading video 2 of 2
[youtube] 8B8ULO4plhg: Downloading webpage


[download] Destination: five little ducklings _ nursery rhymes farmees _ kids songs _ 3d rhymes by Farmees-8B8ULO4plhg.f137.mp4
[download]   1.8% of 828.23MiB at 60.69KiB/s ETA 03:48:477


ERROR: Interrupted by user


256

In [ ]:
# Trying with CC licensed only
os.system("youtube-dl 'ytsearch1:ducklings' --match-filter 'license='Creative Commons Attribution license (reuse allowed)'' ")




In [39]:
# from https://unix.stackexchange.com/questions/230481/how-to-download-portion-of-video-with-youtube-dl-command

import ffmpeg 
import random
# First acquire the actual URL using youtube-dl:
#getURL = os.system("youtube-dl 'https://www.youtube.com/watch?v=MtN1YnoL46Q'")

# Something like this? but how to pass??
# ffmpeg -ss 00:00:15.00 -i getURL -t 00:00:10.00
   
# parameters here https://kkroening.github.io/ffmpeg-python/

# test = (
#     ffmpeg
#     .input(getURL)
#     .trim(start_frame=100, end_frame=250,duration=6)
#     .output("out.mp4")
#     #.global_args('-report')
#     .run()
# )                

import ffmpeg

rand=random.randint(2000,5000)
randy=rand+200

# note ffmpeg.run(stream) flunks out if it involves overwriting... need to add an argument
# the trim command makes for a weird cutting ( long pause then the slice i want )

import ffmpeg
stream = ffmpeg.input('knights.mp4')
stream = ffmpeg.trim(stream, start_frame=500, end_frame=600)
stream = ffmpeg.output(stream, 'outputVeryNewVeryyTrim.mp4')
ffmpeg.run(stream)
stream
# test

print("YASS")

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 4.0.1 (tags/RELEASE_401/final)
  configuration: --prefix=/opt/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55.  5.100 / 55.  5.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'knigh

Error: ffmpeg error (see stderr output for detail)

In [42]:
# ok this works! just horribly verbose

# from https://stackoverflow.com/questions/18444194/cutting-the-videos-based-on-start-and-end-time-using-ffmpeg


#!/usr/bin/env python3

import subprocess

def get_duration(input_video):
    cmd = ["ffprobe", "-i", input_video, "-show_entries", "format=duration",
           "-v", "quiet", "-sexagesimal", "-of", "csv=p=0"]
    return subprocess.check_output(cmd).decode("utf-8").strip()


def main():
    name = "knights.mp4"
    times = []
    times.append(["00:01:10", "00:01:20"])
    times.append(["00:03:00", "00:03:10"])
    # times = [["00:00:00", get_duration(name)]]
    if len(times) == 1:
        time = times[0]
        cmd = ["ffmpeg", "-y", "-i", name, "-ss", time[0], "-to", time[1], "-c:v", "copy", "-c:a", "copy", "output.mp4"]
        subprocess.check_output(cmd)
    else:
        open('concatenate.txt', 'w').close()
        for idx, time in enumerate(times):
            output_filename = f"output{idx}.mp4"
            cmd = ["ffmpeg", "-y", "-i", name, "-ss", time[0], "-to", time[1], "-c:v", "copy", "-c:a", "copy", output_filename]
            subprocess.check_output(cmd)

            with open("concatenate.txt", "a") as myfile:
                myfile.write(f"file {output_filename}\n")

        cmd = ["ffmpeg", "-y", "-f", "concat", "-i", "concatenate.txt", "-c", "copy", "outputss.mp4"]
        output = subprocess.check_output(cmd).decode("utf-8").strip()
        print(output)


if __name__ == "__main__":
    main()

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with clang version 4.0.1 (tags/RELEASE_401/final)
  configuration: --prefix=/opt/miniconda3 --cc=x86_64-apple-darwin13.4.0-clang --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55.  5.100 / 55.  5.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'knigh

In [ ]:
# could also try a different wrapper
# https://ffmpy.readthedocs.io/en/latest/examples.html

In [ ]:
# from https://unix.stackexchange.com/questions/230481/how-to-download-portion-of-video-with-youtube-dl-command

# First acquire the actual URL using youtube-dl:
youtube-dl -g "https://www.youtube.com/watch?v=V_f2QkBdbRI"

# 



In [ ]:
#also need to check out this
# which i downloaded
# https://github.com/kkroening/ffmpeg-python/blob/master/examples/ffmpeg-numpy.ipynb

In [183]:
# still want to get this to work to pull a random date
# can try the other date format

import os

os.system("youtube-dl --date 20052405")





Traceback (most recent call last):
  File "/opt/miniconda3/bin/youtube-dl", line 8, in <module>
    sys.exit(main())
  File "/opt/miniconda3/lib/python3.7/site-packages/youtube_dl/__init__.py", line 475, in main
    _real_main(argv)
  File "/opt/miniconda3/lib/python3.7/site-packages/youtube_dl/__init__.py", line 220, in _real_main
    date = DateRange.day(opts.date)
  File "/opt/miniconda3/lib/python3.7/site-packages/youtube_dl/utils.py", line 3116, in day
    return cls(day, day)
  File "/opt/miniconda3/lib/python3.7/site-packages/youtube_dl/utils.py", line 3103, in __init__
    self.start = date_from_str(start)
  File "/opt/miniconda3/lib/python3.7/site-packages/youtube_dl/utils.py", line 3084, in date_from_str
    return datetime.datetime.strptime(date_str, '%Y%m%d').date()
  File "/opt/miniconda3/lib/python3.7/_strptime.py", line 577, in _strptime_datetime
    tt, fraction, gmtoff_fraction = _strptime(data_string, format)
  File "/opt/miniconda3/lib/python3.7/_strptime.py", line 3

256

In [ ]:
# random script to play with
# https://stackoverflow.com/questions/40445885/no-audio-when-adding-mp3-to-videofileclip-moviepy

def make_video(images):
    image_clips = []
    for img in images:
        if not os.path.exists(img):
            raise FileNotFoundError(img)
        ic = ImageClip(img).set_duration(3)
        image_clips.append(ic)

    video = concatenate(image_clips, method="compose")
    video.set_audio(AudioFileClip("audio.mp3")) 
    video.write_videofile("mp4_with_audio.mp4", fps=60, codec="mpeg4")